<a href="https://www.kaggle.com/code/hilalalpak/online-retail-2010-2011-data-analysis?scriptVersionId=179811572" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import datetime as dt
import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 500)
pd.set_option("display.float_format", lambda x: "%.4f" % x)

df_ = pd.read_excel("/kaggle/input/online-retail-ii-dataset/online_retail_II.xlsx", sheet_name="Year 2010-2011")
df = df_.copy()

In [2]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.5500,17850.0000,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.3900,17850.0000,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.7500,17850.0000,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.3900,17850.0000,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.3900,17850.0000,United Kingdom


In [3]:
df.isnull().sum()

Invoice             0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
Price               0
Customer ID    135080
Country             0
dtype: int64

In [4]:
df.dropna(inplace=True)
df.isnull().sum()

Invoice        0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
Price          0
Customer ID    0
Country        0
dtype: int64

In [5]:
df["StockCode"].nunique()
df["Description"].value_counts()

3684

Description
WHITE HANGING HEART T-LIGHT HOLDER    2070
REGENCY CAKESTAND 3 TIER              1905
JUMBO BAG RED RETROSPOT               1662
ASSORTED COLOUR BIRD ORNAMENT         1418
PARTY BUNTING                         1416
                                      ... 
ANTIQUE RASPBERRY FLOWER EARRINGS        1
WALL ART,ONLY ONE PERSON                 1
GOLD/AMBER DROP EARRINGS W LEAF          1
INCENSE BAZAAR PEACH                     1
PINK BAROQUE FLOCK CANDLE HOLDER         1
Name: count, Length: 3896, dtype: int64

In [6]:
df.groupby("Description").agg({"Quantity" : "sum"}).sort_values(by="Quantity", ascending=False).head(5)

,Quantity
Description,
WORLD WAR 2 GLIDERS ASSTD DESIGNS,53215
JUMBO BAG RED RETROSPOT,45066
ASSORTED COLOUR BIRD ORNAMENT,35314
WHITE HANGING HEART T-LIGHT HOLDER,34147
PACK OF 72 RETROSPOT CAKE CASES,33409


In [7]:
##Remove canceled products

df = df[~df["Invoice"].str.contains("C", na=False)]

In [8]:
##Calculate total price per invoice

df["TotalPrice"] = df["Quantity"] * df["Price"]

* Recency ; Time since the customer's last order date
* Frequency ; Frequency of shopping in a certain period of time
* Monetary ; Total spending by the customer

In [9]:
##Recency
df["InvoiceDate"].max()
today_date = dt.datetime(year=2011, month=12, day=11)

rfm = df.groupby("Customer ID").agg({"InvoiceDate": lambda x: (today_date - x.max()).days,
                                           "Invoice": lambda x: x.nunique(),
                                           "TotalPrice": lambda x: x.sum()})
rfm.columns = ["recency", "frequency", "monetary"]
rfm.describe().T

Timestamp('2011-12-09 12:50:00')

,count,mean,std,min,25%,50%,75%,max
recency,4339.0000,93.0415,100.0078,1.0000,18.0000,51.0000,142.5000,374.0000
frequency,4339.0000,4.2720,7.7055,1.0000,1.0000,2.0000,5.0000,210.0000
monetary,4339.0000,2053.7972,8988.2478,0.0000,307.2450,674.4500,1661.6400,280206.0200


In [10]:
##We don't want the Monetary value to be 0, fly it
rfm = rfm[rfm["monetary"] > 0]
rfm.describe().T

,count,mean,std,min,25%,50%,75%,max
recency,4338.0000,93.0595,100.0123,1.0000,18.0000,51.0000,142.7500,374.0000
frequency,4338.0000,4.2727,7.7062,1.0000,1.0000,2.0000,5.0000,210.0000
monetary,4338.0000,2054.2706,8989.2299,3.7500,307.4150,674.4850,1661.7400,280206.0200


In [11]:
rfm["recency_score"] = pd.qcut(rfm["recency"], 5, labels=[5, 4, 3, 2, 1])
rfm["frequency_score"] = pd.qcut(rfm["frequency"].rank(method="first"), 5, labels=[1, 2, 3, 4, 5])
rfm["monetary_score"] = pd.qcut(rfm["monetary"], 5, labels=[1, 2, 3, 4, 5])
rfm["RF_SCORE"] = (rfm["recency_score"].astype(str) + rfm["frequency_score"].astype(str))
rfm.head()

,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RF_SCORE
Customer ID,,,,,,,
12346.0000,326,1,77183.6000,1,1,5,11
12347.0000,3,7,4310.0000,5,5,5,55
12348.0000,76,4,1797.2400,2,4,4,24
12349.0000,19,1,1757.5500,4,1,4,41
12350.0000,311,1,334.4000,1,1,2,11


> We used the "rank" method because there are many repeating frequencies and the same values fall in each range. With "Rank"; "Assign the first class you see to the first class"

In [12]:
seg_map = {r'[1-2][1-2]': 'Hibernating',
    r'[1-2][3-4]': 'At_Risk',
    r'[1-2]5': 'Cant_Loose',
    r'3[1-2]': 'About_to_Sleep',
    r'33': 'Need_Attention',
    r'[3-4][4-5]': 'Loyal_Customers',
    r'41': 'Promising',
    r'51': 'New_Customers',
    r'[4-5][2-3]': 'Potential_Loyalists',
    r'5[4-5]': 'Champions'}

In [13]:
rfm["segment"] = rfm["RF_SCORE"].replace(seg_map, regex=True)
rfm.head(10)

,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RF_SCORE,segment
Customer ID,,,,,,,,
12346.0000,326,1,77183.6000,1,1,5,11,Hibernating
12347.0000,3,7,4310.0000,5,5,5,55,Champions
12348.0000,76,4,1797.2400,2,4,4,24,At_Risk
12349.0000,19,1,1757.5500,4,1,4,41,Promising
12350.0000,311,1,334.4000,1,1,2,11,Hibernating
12352.0000,37,8,2506.0400,3,5,5,35,Loyal_Customers
12353.0000,205,1,89.0000,1,1,1,11,Hibernating
12354.0000,233,1,1079.4000,1,1,4,11,Hibernating
12355.0000,215,1,459.4000,1,1,2,11,Hibernating
